## Import mstrio

In [52]:
from mstrio.connection import Connection
from mstrio.admin.application import Environment, Application, compare_application_settings

## Session 

Connection Information. And create session.

In [ ]:
base_url = "https://env-248739.customer.cloud.microstrategy.com/MicroStrategyLibrary/api"
mstr_username = "mstr"
mstr_password = ""
project_id = "B7CA92F04B9FAE8D941C3E9B7E0CD754"
conn = Connection(base_url, mstr_username, mstr_password, project_id=project_id)

Connection to MicroStrategy Intelligence Server has been established.


Get environment object

In [64]:
env = Environment(connection=conn)

## List all applications

In [69]:
all_applications = env.list_applications()

In [70]:
all_applications

[Application(connection, name='Consolidated Education Project', id='CE52831411E696C8BD2F0080EFD5AF44'),
 Application(connection, name='Hierarchies Project', id='B3FEE61A11E696C8BD0F0080EFC58F44'),
 Application(connection, name='Human Resources Analysis Module', id='4BAE16A340B995CAD24193AA3AC15D29'),
 Application(connection, name='MicroStrategy Tutorial', id='B7CA92F04B9FAE8D941C3E9B7E0CD754'),
 Application(connection, name='Platform Analytics', id='4452A9581744B4A1CBC7FC966D298224'),
 Application(connection, name='Relationships Project', id='4C09350211E69712BAEE0080EFB56D41'),
 Application(connection, name='Test Project 3', id='9D99957974410B8A1A148A9706006560')]

In [57]:
test = all_applications[-1]

In [44]:
test.unload()

'Test Project 3' unloaded on node 'env-248739laiouse1'.


In [49]:
test.unregister()

Application will not load on startup


In [58]:
test.to_dataframe()

,value
id,9D99957974410B8A1A148A9706006560
name,Test Project 3
description,Test Project 3
date_created,2021-04-21T02:45:12.000+0000
date_modified,2021-04-21T02:45:23.000+0000
owner,"{'name': 'MSTR User', 'id': '7FC05A65473CE2FD845CE6A1D3F13233'}"


### Create a project

In [65]:
app = env.create_application(name="Test Project 4", force=True)

Application 'Test Project 4' successfully created.
Application object named: 'Test Project 4' with ID: '653DCCC0CE4B1E9E8F4BF0872FB7080A'


In [66]:
app

Application(connection, name='Test Project 4', id='653DCCC0CE4B1E9E8F4BF0872FB7080A')

## Get project configuration

In [39]:
res = conn.session.get(url=conn.base_url + "/api/v2/projects/59995B6D461F6FFEC04148B962AC8E95/settings")
print(res.status_code)
print(res.json())

400
{'code': 'E100001', 'message': "The project id '59995B6D461F6FFEC04148B962AC8E95' is invalid", 'ticketId': 'c08432a264444603aef3189e66bc1d75'}


## Update Project Configuration

In [93]:
res = conn.session.patch(url=conn.base_url + "/api/v2/projects/59995B6D461F6FFEC04148B962AC8E95/settings",
                         headers={"Content-Type":"application/json"},
                         data='''
   {
     "maxElementCacheMemoryConsumption": {
        "value": 1024
    },
    "maxObjectCacheMemoryConsumption": {
        "value": 2048
    },
    "reportCacheSwapFactor": {
        "value": 2
    },
    "reportCacheLoadOnStart": {
      "value": true
    },
    "maxRWDCacheMemoryConsumption": {
      "value": 1024
    },
    "maxReportCacheCount": {
        "value": 100
    },
    "maxRWDCacheCount": {
        "value": 100
    },
    "reportCacheLifeTime": {
      "value": -1
    }
}''')

## Get the information of the cluster nodes

In [94]:
res = conn.session.get(url=conn.base_url + "/api/monitors/iServer/nodes")
print(res.status_code)
print(res.json())

200
{'nodes': [{'name': 'TYCW4776S', 'port': 34952, 'status': 'running', 'load': 2, 'projects': [{'id': 'B7CA92F04B9FAE8D941C3E9B7E0CD754', 'name': 'MicroStrategy Tutorial', 'status': 'loaded'}, {'id': '30F8F5E54328E34EBD93E0AD286032EF', 'name': 'Platform Analytics', 'status': 'loaded'}, {'id': 'D5835B164FB8521340B51DB2CA5F7CDC', 'name': 'Test Project', 'status': 'unloaded'}, {'id': '142ECEEB4BBAE6F74D8F3AAB2BA2B82D', 'name': 'Test Project2', 'status': 'unloaded'}, {'id': '59995B6D461F6FFEC04148B962AC8E95', 'name': 'Test Project 3', 'status': 'loaded'}], 'default': True}]}


## Unload a project

In [31]:
res = conn.session.patch(url=conn.base_url + "/api/monitors/iServer/nodes/TYCW4776S/projects/142ECEEB4BBAE6F74D8F3AAB2BA2B82D",
                         headers={"Content-Type":"application/json"},
                         data='''
    {"operationList": [
        {
            "op": "replace",
            "path": "/status",
            "value": "unloaded"
        }
    ]}''')

In [32]:
print(res.status_code)
print(res.json())

202
{'project': {'name': 'Test Project2', 'id': '142ECEEB4BBAE6F74D8F3AAB2BA2B82D', 'status': 'pending'}}


In [55]:
res = conn.session.get(url=conn.base_url + "/api/monitors/iServer/nodes")
print(res.status_code)
print(res.json())

200
{'nodes': [{'name': 'TYCW4776S', 'port': 34952, 'status': 'running', 'load': 2, 'projects': [{'id': 'B7CA92F04B9FAE8D941C3E9B7E0CD754', 'name': 'MicroStrategy Tutorial', 'status': 'loaded'}, {'id': '30F8F5E54328E34EBD93E0AD286032EF', 'name': 'Platform Analytics', 'status': 'loaded'}, {'id': 'D5835B164FB8521340B51DB2CA5F7CDC', 'name': 'Test Project', 'status': 'loaded'}, {'id': '142ECEEB4BBAE6F74D8F3AAB2BA2B82D', 'name': 'Test Project2', 'status': 'unloaded'}], 'default': True}]}


## Get project startup settings

In [73]:
res = conn.session.get(url=conn.base_url + "/api/projects/settings/onStartup",
                         headers={"Content-Type":"application/json"}
                         )

In [74]:
print(res.status_code)
print(res.json())

200
{'projects': {'142ECEEB4BBAE6F74D8F3AAB2BA2B82D': {'name': 'Test Project2', 'id': '142ECEEB4BBAE6F74D8F3AAB2BA2B82D', 'nodes': []}, '30F8F5E54328E34EBD93E0AD286032EF': {'name': 'Platform Analytics', 'id': '30F8F5E54328E34EBD93E0AD286032EF', 'nodes': ['TYCW4776S.corp.microstrategy.com']}, 'B7CA92F04B9FAE8D941C3E9B7E0CD754': {'name': 'MicroStrategy Tutorial', 'id': 'B7CA92F04B9FAE8D941C3E9B7E0CD754', 'nodes': ['TYCW4776S.corp.microstrategy.com']}, 'D5835B164FB8521340B51DB2CA5F7CDC': {'name': 'Test Project', 'id': 'D5835B164FB8521340B51DB2CA5F7CDC', 'nodes': []}}}


## Change project startup settings.  (Unregister Project)

In [75]:
res = conn.session.patch(url=conn.base_url + "/api/projects/settings/onStartup",
                         headers={"Content-Type":"application/json"},
                         data='''{ "operationList":[ {
                                 "op":"replace",
                                 "path":"/projects/D5835B164FB8521340B51DB2CA5F7CDC/nodes",
                                 "value": []
                              }]}
                              '''
                         )

In [76]:
print(res.status_code)
print(res.json())

200
{'projects': {'142ECEEB4BBAE6F74D8F3AAB2BA2B82D': {'name': 'Test Project2', 'id': '142ECEEB4BBAE6F74D8F3AAB2BA2B82D', 'nodes': []}, '30F8F5E54328E34EBD93E0AD286032EF': {'name': 'Platform Analytics', 'id': '30F8F5E54328E34EBD93E0AD286032EF', 'nodes': ['TYCW4776S.corp.microstrategy.com']}, 'B7CA92F04B9FAE8D941C3E9B7E0CD754': {'name': 'MicroStrategy Tutorial', 'id': 'B7CA92F04B9FAE8D941C3E9B7E0CD754', 'nodes': ['TYCW4776S.corp.microstrategy.com']}, 'D5835B164FB8521340B51DB2CA5F7CDC': {'name': 'Test Project', 'id': 'D5835B164FB8521340B51DB2CA5F7CDC', 'nodes': []}}}


## Get Server Configuration

In [98]:
res = conn.session.get(url=conn.base_url + "/api/v2/iserver/settings",
                         headers={"Content-Type":"application/json"}
                         )
# print(res.status_code)
print(res.json())

{'maxUsedVirtualByte': {'value': 99}, 'importSAPUser': {'value': False}, 'maxInboxMsgLifeTime': {'value': -1}, 'hashIterations': {'value': 10000}, 'tokenTTL': {'value': 1440}, 'loadBalanceMode': {'value': False}, 'maxLoginAttempt': {'value': -1}, 'maxUserConnectionIdleTime': {'value': 7200}, 'syncGroupWithSAPRoles': {'value': False}, 'maxInteractiveJobsPerServer': {'value': -1}, 'pastPasswordRemembered': {'value': 0}, 'enableTokenAsymmetricEncryption': {'value': False}, 'pdfMaxMemoryConsumption': {'value': 536870912}, 'historyListRunningStatus': {'value': False}, 'userAffinityFlag': {'value': False}, 'enableBackgroundDiExecution': {'value': False}, 'minLowerCasePasswordChars': {'value': 0}, 'statsLogging2ProjectGUID': {'value': '00000000000000000000000000000000'}, 'sessionRecoveryPath': {'value': '.\\Inbox\\cloud_10s'}, 'memoryRequestIdleTime': {'value': 600}, 'serverSchedulerFlag': {'value': True}, 'synchUserAtLogon': {'value': False}, 'memoryReservePercentage': {'value': 10}, 'minCha

## Change Server Configuration

In [101]:
res = conn.session.patch(url=conn.base_url + "/api/v2/iserver/settings",
                         headers={"Content-Type":"application/json"},
                         data='''{
                             "maxUserConnectionIdleTime": {
                                "value": 7200
                             }
                             }'''
                         )

print(res.json())

{'maxUsedVirtualByte': {'value': 99}, 'importSAPUser': {'value': False}, 'maxInboxMsgLifeTime': {'value': -1}, 'hashIterations': {'value': 10000}, 'tokenTTL': {'value': 1440}, 'loadBalanceMode': {'value': False}, 'maxLoginAttempt': {'value': -1}, 'maxUserConnectionIdleTime': {'value': 7200}, 'syncGroupWithSAPRoles': {'value': False}, 'maxInteractiveJobsPerServer': {'value': -1}, 'pastPasswordRemembered': {'value': 0}, 'enableTokenAsymmetricEncryption': {'value': False}, 'pdfMaxMemoryConsumption': {'value': 536870912}, 'historyListRunningStatus': {'value': False}, 'userAffinityFlag': {'value': False}, 'enableBackgroundDiExecution': {'value': False}, 'minLowerCasePasswordChars': {'value': 0}, 'statsLogging2ProjectGUID': {'value': '00000000000000000000000000000000'}, 'sessionRecoveryPath': {'value': '.\\Inbox\\cloud_10s'}, 'memoryRequestIdleTime': {'value': 600}, 'serverSchedulerFlag': {'value': True}, 'synchUserAtLogon': {'value': False}, 'memoryReservePercentage': {'value': 10}, 'minCha

## Close Session

In [77]:
conn.close()

Connection to MicroStrategy Intelligence Server has been closed
